# Baixando a implementação do YOLO 5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 11793, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 11793 (delta 0), reused 1 (delta 0), pack-reused 11789
Receiving objects: 100% (11793/11793), 11.39 MiB | 17.36 MiB/s, done.
Resolving deltas: 100% (8160/8160), done.
/content/yolov5
     |████████████████████████████████| 596 kB 4.9 MB/s 
     |████████████████████████████████| 1.1 MB 8.4 MB/s 
     |████████████████████████████████| 138 kB 42.2 MB/s 
     |████████████████████████████████| 178 kB 46.4 MB/s 
     |████████████████████████████████| 67 kB 1.0 MB/s 
     |████████████████████████████████| 145 kB 44.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.

# Baixando Dataset criado no Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

os.environ["DATASET_DIRECTORY"] = "/content/datasets"

rf = Roboflow(api_key="K3I9PA5xYKDA0RFSDjrB")
project = rf.workspace("guilherme-piveti-filho").project("augmented-equilibrado-rivet")
dataset = project.version(1).download("yolov5")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/Augmented-Equilibrado-Rivet-1 in yolov5pytorch:: 100%|██████████| 438/438 [00:00<00:00, 1001.60it/s]


# Treinando o modelo

Treinando o modelo por 1000 épocas

In [ ]:
!python train.py --img 416 --batch 16 --epochs 1000 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/datasets/Augmented-Equilibrado-Rivet-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1000, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-62-ga0a4adf torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, h

# Avaliando o modelo


In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images --save-txt --save-conf --max-det 1

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/datasets/Augmented-Equilibrado-Rivet-1/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-62-ga0a4adf torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/32 /content/datasets/Augmented-Equilibrado-Rivet-1/test/images/f--2-_bmp_jpg.rf.b28917ebf95b2db879ddbc956bff0d0e.jpg: 416x416 1 Fract, Done. (0.026s)
image 2/32 /content/datasets/Augmented-Equilibrado-Rivet-1/test/images/f--21-_bmp_jpg.rf.dffcbbf52b309bb7a3eb379b8b28c604.jpg: 416x416 1 Fract, Done. (0.026s)
image 3/32 /content/dat

# Calculando matriz confusão

In [ ]:
import glob

false_frat = 0
true_frat = 0
false_nfrat = 0
true_nfrat = 0

for imageName in glob.glob('/content/yolov5/runs/detect/exp/labels/*.txt'):
    if(imageName.split('/')[-1][:2]=='nf'):
      with open(imageName) as f:
        contents = f.readlines()
        if(contents[0][0]=='0'):
          false_frat += 1
        else:
          true_nfrat += 1
    else:
      with open(imageName) as f:
        contents = f.readlines()
        if(contents[0][0]=='0'):
          true_frat += 1
        else:
          false_nfrat += 1

print(true_frat)
print(false_nfrat)
print(false_frat)
print(true_nfrat)

7
3
2
20


# Exibindo o resultado

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/yolov5/runs/detect/exp/*.jpg'):
    print(imageName)
    display(Image(filename=imageName))
    print("\n")

# Salvando o modelo

In [ ]:
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>